### merge file 2023/12/29 -  2024/2/2|

In [31]:
import os
import pandas as pd

# Define the target directories
target1 = '../Satun-Phang-Nga/final-Phang-Nga-SunriseSunset/'
target2 = '../Satun-Phang-Nga/23-final-Phang-Nga-SunriseSunset/'
output_dir = '../Satun-Phang-Nga/forcasting/'
os.makedirs(output_dir, exist_ok=True)

# Get a list of files in both directories
files1 = sorted(os.listdir(target1))[0:5]
files2 = sorted(os.listdir(target2))


# Loop through the files in target1 and target2 assuming they are in the same order
for file1, file2 in zip(files1, files2):

    # Read df1 from target1
    file_path1 = os.path.join(target1, file1)
    df1 = pd.read_excel(file_path1)
    df1 = df1[:33]  # Select specific rows (example)

    # Read df2 from target2
    file_path2 = os.path.join(target2, file2)
    df2 = pd.read_excel(file_path2)
    df2 = df2.iloc[-2:]  

    # Combine df1 and df2
    combined_df = pd.concat([df2, df1], ignore_index=True)

    # Save the combined DataFrame to a new Excel file
    file1 = file1.split('-')[:-1]
    file1 = '-'.join(file1)+'.xlsx'
    print(file1)
    output_file = os.path.join(output_dir, file1)
    combined_df.to_excel(output_file, index=False)
    print(f"Combined Excel file saved at: {output_file}")

print("All files processed and combined successfully.")


1-2024-02-02-Na-Toei-Thai-Mueang-Phang-nga.xlsx
Combined Excel file saved at: ../Satun-Phang-Nga/forcasting/1-2024-02-02-Na-Toei-Thai-Mueang-Phang-nga.xlsx
2-2024-02-02-Na-Toei-Thai-Mueang-Phang-nga.xlsx
Combined Excel file saved at: ../Satun-Phang-Nga/forcasting/2-2024-02-02-Na-Toei-Thai-Mueang-Phang-nga.xlsx
3-2024-02-02-Na-Toei-Thai-Mueang-Phang-nga.xlsx
Combined Excel file saved at: ../Satun-Phang-Nga/forcasting/3-2024-02-02-Na-Toei-Thai-Mueang-Phang-nga.xlsx
4-2024-02-02-Na-Toei-Thai-Mueang-Phang-nga.xlsx
Combined Excel file saved at: ../Satun-Phang-Nga/forcasting/4-2024-02-02-Na-Toei-Thai-Mueang-Phang-nga.xlsx
5-2024-02-02-Na-Toei-Thai-Mueang-Phang-nga.xlsx
Combined Excel file saved at: ../Satun-Phang-Nga/forcasting/5-2024-02-02-Na-Toei-Thai-Mueang-Phang-nga.xlsx
All files processed and combined successfully.


### lable

In [32]:
import os
import pandas as pd

# Define the target directory and the label list
target = '../Satun-Phang-Nga/forcasting/'
label = [1, 0, 2, 3, 4]

# Loop through each file in the target directory
for i, file in enumerate(os.listdir(target)):
    file_path = os.path.join(target, file)
    df = pd.read_excel(file_path)
    
    num_rows = len(df)
    
    # Initialize a list to store the 'disease' values
    disease_values = [0] * num_rows
    
    # Set the initial label value
    current_label = label[i]
    
    # Assign values starting from the end of the DataFrame
    for j in range(num_rows):
        # Calculate the label for the current row
        disease_values[-(j+1)] = current_label
        
        # Update the label value every 7 rows
        if (j + 1) % 7 == 0:
            current_label = max(current_label - 1, 0)
    
    # Add the 'disease' column to the DataFrame
    df['disease'] = disease_values
    
    # Save the updated DataFrame back to the original file
    df.to_excel(file_path, index=False)
    print(f"Added 'disease' column to file: {file_path}")

print("All files processed and updated successfully.")


Added 'disease' column to file: ../Satun-Phang-Nga/forcasting/1-2024-02-02-Na-Toei-Thai-Mueang-Phang-nga.xlsx
Added 'disease' column to file: ../Satun-Phang-Nga/forcasting/2-2024-02-02-Na-Toei-Thai-Mueang-Phang-nga.xlsx
Added 'disease' column to file: ../Satun-Phang-Nga/forcasting/3-2024-02-02-Na-Toei-Thai-Mueang-Phang-nga.xlsx
Added 'disease' column to file: ../Satun-Phang-Nga/forcasting/4-2024-02-02-Na-Toei-Thai-Mueang-Phang-nga.xlsx
Added 'disease' column to file: ../Satun-Phang-Nga/forcasting/5-2024-02-02-Na-Toei-Thai-Mueang-Phang-nga.xlsx
All files processed and updated successfully.


In [37]:
import requests
import os
import pandas as pd

# Define the URL and parameters for the API request
url = "https://power.larc.nasa.gov/api/temporal/daily/point"
parameters = ['GWETPROF', 'ALLSKY_SFC_SW_DWN', 'ALLSKY_SFC_LW_DWN', 'PS', 'PW', 'RH2M']
target = '../Satun-Phang-Nga/forcasting/'
output_dir = '../Satun-Phang-Nga/forcasting2/'
os.makedirs(output_dir, exist_ok=True)

# Process each file in the target directory
for i, file in enumerate(os.listdir(target)):
    file_path = os.path.join(target, file)
    df_existing = pd.read_excel(file_path)
    
    # Print columns to verify names
    print(f"Columns in file {file}: {df_existing.columns.tolist()}")
    
    # Ensure that 'date', 'latitude', and 'longitude' columns are present
    if 'date' not in df_existing.columns or 'latitude' not in df_existing.columns or 'longitude' not in df_existing.columns:
        print(f"Missing one of the required columns in file {file}")
        continue
    
    # Convert 'date' column to datetime
    df_existing['date'] = pd.to_datetime(df_existing['date'])
    
    start = df_existing['date'].min().strftime('%Y%m%d')
    end = df_existing['date'].max().strftime('%Y%m%d')
    latitude = df_existing['latitude'].iloc[0]
    longitude = df_existing['longitude'].iloc[0]
    community = 'AG'
    
    params = {
        "parameters": ','.join(parameters),
        "start": start,
        "end": end,
        "latitude": latitude,
        "longitude": longitude,
        "community": community
    }

    response = requests.get(url, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        if 'properties' in data:
            # Extract the parameter data
            parameters_data = data['properties']['parameter']
            
            # Create a DataFrame to hold all parameters
            df_weather = pd.DataFrame()
            
            # Iterate through each parameter and add it to the DataFrame
            for param in parameters:
                if param in parameters_data:
                    param_data = parameters_data[param]
                    df_param = pd.DataFrame(list(param_data.items()), columns=['date', param])
                    df_param['date'] = pd.to_datetime(df_param['date'], format='%Y%m%d')
                    
                    # Merge with existing DataFrame
                    if df_weather.empty:
                        df_weather = df_param
                    else:
                        df_weather = pd.merge(df_weather, df_param, on='date', how='outer')
            
            # Merge the new weather data with the existing file data
            df_final = pd.merge(df_existing, df_weather, on='date', how='outer')
            
            # Define output file path
            output_file = os.path.join(output_dir, f"{file}")
            # Save DataFrame to Excel
            df_final.to_excel(output_file, index=False)
            print(f"Saved updated data for file {file} to {output_file}")
        else:
            print(f"No 'properties' in data for file {file}")
    else:
        # Print detailed error information
        print(f"Error: {response.status_code} for file {file}")
        print("Response content:", response.text)


Columns in file 1-2024-02-02-Na-Toei-Thai-Mueang-Phang-nga.xlsx: ['date', 'latitude', 'longitude', 'uvb', 'e', 'stl1', 'sp', 'tp', 'd2m', 'minTemp', 'meanTemp', 'maxTemp', 'RH', 'WS', 'sunrise', 'sunset', 'durationOfDay', 'disease']
Saved updated data for file 1-2024-02-02-Na-Toei-Thai-Mueang-Phang-nga.xlsx to ../Satun-Phang-Nga/forcasting2/1-2024-02-02-Na-Toei-Thai-Mueang-Phang-nga.xlsx
Columns in file 2-2024-02-02-Na-Toei-Thai-Mueang-Phang-nga.xlsx: ['date', 'latitude', 'longitude', 'uvb', 'e', 'stl1', 'sp', 'tp', 'd2m', 'minTemp', 'meanTemp', 'maxTemp', 'RH', 'WS', 'sunrise', 'sunset', 'durationOfDay', 'disease']
Saved updated data for file 2-2024-02-02-Na-Toei-Thai-Mueang-Phang-nga.xlsx to ../Satun-Phang-Nga/forcasting2/2-2024-02-02-Na-Toei-Thai-Mueang-Phang-nga.xlsx
Columns in file 3-2024-02-02-Na-Toei-Thai-Mueang-Phang-nga.xlsx: ['date', 'latitude', 'longitude', 'uvb', 'e', 'stl1', 'sp', 'tp', 'd2m', 'minTemp', 'meanTemp', 'maxTemp', 'RH', 'WS', 'sunrise', 'sunset', 'durationOfDa

In [38]:
import os
import pandas as pd
import numpy as np

target = '../Satun-Phang-Nga/forcasting2/'


gamma = 0.665 * 10**-3


def calculate_delta(Tmean):
    return (4098 * (0.6108 * np.exp((17.27 * Tmean) / (Tmean + 237.3)))) / (Tmean + 237.3)**2


def calculate_saturation_vapor_pressure(Tmean):
    return 0.6108 * np.exp((17.27 * Tmean) / (Tmean + 237.3))


def calculate_actual_vapor_pressure(RH, Tmean):
    e_s = calculate_saturation_vapor_pressure(Tmean)
    return (RH / 100) * e_s


def calculate_Net_Radiation(Rs, Rl):
    alpha = 0.23
    Rl = (Rl*86400)/(10**6)
    Rn = Rs * (1 - alpha) - Rl
    return Rn


for file in os.listdir(target):
    file_path = os.path.join(target, file)
    df = pd.read_excel(file_path)

    Tmean = df['meanTemp']
    RH = df['RH']
    Rs = df['ALLSKY_SFC_SW_DWN']
    Rl = df['ALLSKY_SFC_LW_DWN']
    u = df['WS']
    P = df['PS']

    delta = calculate_delta(Tmean)
    e_s = calculate_saturation_vapor_pressure(Tmean)
    e_a = calculate_actual_vapor_pressure(RH, Tmean)
    Rn = calculate_Net_Radiation(Rs, Rl)

    G = 0
    ETo = (0.408 * delta * (Rn - G) + gamma * (900 / (Tmean + 273))
           * u * (e_s - e_a)) / (delta + gamma * (1 + 0.34 * u))
    df['ETo'] = ETo

    # Calculate WE and WD
    df['WE'] = P - ETo
    df['WD'] = ETo - P

    output_path = os.path.join(target, file)
    df.to_excel(output_path, index=False)